# Imports

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q bitsandbytes einops


from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoModelForSeq2SeqLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer #supervised finetuning
from huggingface_hub import login

  error: subprocess-exited-with-error
  
  × git version did not run successfully.
  │ exit code: 1
  ╰─> [2 lines of output]
      xcrun: error: invalid active developer path (/Library/Developer/CommandLineTools), missing xcrun at: /Library/Developer/CommandLineTools/usr/bin/xcrun
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git version did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


ModuleNotFoundError: No module named 'datasets'

In [3]:
model_name = "EleutherAI/polyglot-ko-1.3b"

# Dataset

In [5]:
def prompt_instruction_format(sample):
  return f""" ### Instruction:
    Use the taks below and the input given to write the response:

    ### Task:
    Give response to the input dialouge

    ### Input:
    {sample['Dialogue']}

    ### Reponse:
    {sample['Response']}
    """

In [6]:
from datasets import Dataset
import pandas as pd
test = pd.read_excel('test.xlsx')
dataset_test = Dataset.from_pandas(test)

train = pd.read_excel('train.xlsx')
dataset= Dataset.from_pandas(train)

# dataset = {'train': dataset_train, 'test': dataset_test}

ModuleNotFoundError: No module named 'datasets'

In [ ]:
from transformers import AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXConfig

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPTNeoXForCausalLM.from_pretrained(model_name)

inputs = tokenizer("진로가 상당히 걱정돼", return_tensors="pt")

# Exclude token_type_ids from the inputs
# Only pass input_ids and attention_mask to the model
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

outputs = model(input_ids=input_ids, attention_mask=attention_mask)

prediction_logits = outputs.logits


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token #end of sentence tokenizer
# tokenizer.padding_side = "right"

from transformers import  TrainingArguments

trainingArgs = TrainingArguments(
    output_dir = '/output',
    num_train_epochs = 1,
    per_device_train_batch_size = 4,
    save_strategy= "epoch",
    learning_rate = 2e-4,
    optim="paged_adamw_32bit",
    logging_steps=1,
)
# trainingArgs = TrainingArguments(
#     output_dir = '/output',
#     num_train_epochs=100,
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=2,
#     gradient_checkpointing=True,
#     optim="paged_adamw_32bit", # from the QLoRA paper
#     logging_steps=1,
#     save_strategy="epoch",
#     learning_rate=2e-4,
#     max_grad_norm=0.3,
#     warmup_ratio=0.03,
#     lr_scheduler_type="constant",
#     disable_tqdm=True
# )

In [ ]:
#create the trainer
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM" #predicting the next token in the sequence
)

max_seq_length = 2048

trainer = SFTTrainer(
    model= model,
    train_dataset=dataset.select(range(200)),
    peft_config = peft_config,
    max_seq_length=max_seq_length,
    tokenizer = tokenizer,
    packing=True,
    formatting_func = prompt_instruction_format,
    args= trainingArgs,
)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 33.06 MiB is free. Process 435125 has 14.71 GiB memory in use. Of the allocated memory 14.51 GiB is allocated by PyTorch, and 81.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.save_model()

In [ ]:
from peft import AutoPeftModelForCausalLM

# Load the trained model
model = AutoPeftModelForCausalLM.from_pretrained(
    trainingArgs.output_dir,  # Ensure this is the correct path
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)


In [ ]:
# No need to reinitialize the tokenizer if it hasn't changed
# tokenizer = AutoTokenizer.from_pretrained(trainingArgs.output_dir)

# Get a sample from the dataset
sample = dataset_test[5]

# Format the prompt
prompt = prompt_instruction_format(sample)

# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids
# If you're using a GPU, you can uncomment the following line
# input_ids = input_ids.cuda()

# Make sure the model and inputs are on the GPU
model = model.to('cuda')
input_ids = input_ids.to('cuda')

# Generate the model's response
outputs = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    do_sample=True,
    top_p=0.9,
    temperature=0.5
)

In [ ]:
# Decode the generated response
generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Prompt:\n{prompt}\n")
print(f"-------------")
print(f"Generated Response:\n{generated_response}")
print(f"-------------")
print(f"Ground truth:\n{sample['Response']}")

# response from untuned model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the untuned model and tokenizer
untuned_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure that 'prompt' is a properly formatted string
# Replace 'prompt[0]' with 'prompt' if 'prompt' is not a list
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

# If you're using a GPU, uncomment the following line
# input_ids = input_ids.cuda()

# Generate the response
outputs = untuned_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=0.5)

# Decode the generated output
generated_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the results
print(f"Prompt:\n{prompt}\n")
print(f"-------------")
print(f"Generated instruction:\n{generated_output}")
print(f"-------------")
print(f"Ground truth:\n{sample['Response']}")
